In [1]:
from common import *

/home/ec2-user/anaconda3/lib/python3.6/site-packages/pandas/core/common.py:701: RuntimeWarning: divide by zero encountered in log
  return func(obj, *args, **kwargs)


In [2]:
with open(top_dir + 'data/chance/days_order.txt', 'r') as file:
    days = pd.Series([int(x) for x in file.read().splitlines()], name='Day')
with open(top_dir + 'data/chance/days_order.txt', 'r') as file:
    daysr = pd.Series([int(x) for x in file.read().splitlines()][::-1], name='Day')
n_days = len(days)
days_z = dg.Day.unique()

In [3]:
n_days

258

In [4]:
if __name__ == '__main__':
    days_a, days_b, days_c = days[:n_days // 3], days[n_days // 3:(2 * n_days) // 3], days[(2 * n_days) // 3:]
    print(len(days_a), len(days_b), len(days_c))

86 86 86


In [5]:
def split2(dh, ho=True, repeat=1, reverse=False):
    dord = daysr if reverse else days
    if not isinstance(dh.columns, pd.MultiIndex) and 'Day' in dh:
        index = dh.Day
    elif isinstance(dh, pd.Series) and dh.name == 'Day':
        index = dh
    else:
        assert dh.index.name == 'Day'
        index = dh.index
    if ho:
        tc, ho = dord[:176], dord[176:]
    else:
        tc = dord
    t0c1 = [(tc.index // (1 << i) % 2).values.astype(bool) for i in range(repeat)]
    #
    indexify = lambda a: index.isin(set(a)) if isinstance(index, pd.Series) else pd.Series(index.isin(set(a)), index=index)
    s = O(type=2, repeat=repeat, n_folds=2)
    s.tc = indexify(tc)
    s.trcv = [(indexify(tc[~x]), indexify(tc[x])) for x in t0c1]
    if repeat == 1:
        s.tr = indexify(tc[~t0c1[0]])
        s.cv = indexify(tc[t0c1[0]])
    if ho is not None and ho is not False and ho is not 0:
        s.ho = indexify(ho)
    s.lb = indexify(days_z)
    s.aw = dh.Day > -1
    return s

In [6]:
def split3(dh, ho=True, repeat=1, reverse=False):
    dord = daysr if reverse else days
    if not isinstance(dh.columns, pd.MultiIndex) and 'Day' in dh:
        index = dh.Day
    elif isinstance(dh, pd.Series) and dh.name == 'Day':
        index = dh
    else:
        assert dh.index.name == 'Day'
        index = dh.index
    if ho:
        tc, ho = dord[:180], dord[180:]
    else:
        tc = dord
    tc3 = [(tc.index // 3 ** i % 3).values for i in range(repeat)]
    #
    indexify = lambda a: index.isin(set(a)) if isinstance(index, pd.Series) else pd.Series(index.isin(set(a)), index=index)
    s = O(type=3, repeat=repeat, n_folds=3)
    s.tc = indexify(tc)
    s.trcv = [] #(indexify(tc[x < 2]), indexify(tc[x == 2])) for x in tc3]
    for r in range(repeat):
        s.trcv += [(indexify(tc[tc3[r] != i]), indexify(tc[tc3[r] == i])) for i in range(3)]
    s.ho = indexify(ho)
    s.lb = indexify(days_z)
    s.aw = dh.Day > -1
    return s

In [163]:
def split_label(index, n_folds, rep=0):
    assert len(index) >= n_folds ** rep, 'Deterministic cv-splitting algorithm repeated too much on same random order'
    assert not isinstance(index, pd.Series) and not isinstance(index, pd.DataFrame)
    guy = pd.DataFrame({'Order': np.arange(len(index))}) #'Index': index
    guy['Label'] = guy.Order // n_folds ** rep % n_folds
    guy = guy.chain.sort_values(['Label', 'Order'], inplace=True)
    guy['Label'] = (np.arange(len(index)) * n_folds) // len(index)
    guy = guy.chain.sort_values('Order', inplace=True)
    #assert all(index == guy.Index)
    return pd.Series(guy.Label.values, index=index)

In [219]:
def splitf(dh, n_folds, ho=258-180, repeat=1, reverse=False):
    hoint = int(ho)
    dord = daysr if reverse else days
    if not isinstance(dh.columns, pd.MultiIndex) and 'Day' in dh:
        index = dh.Day
    elif isinstance(dh, pd.Series) and dh.name == 'Day':
        index = dh
    else:
        assert dh.index.name == 'Day'
        index = dh.index
    tc, ho = dord[:258 - ho], dord[258 - ho:]
    assert len(tc) % n_folds == 0, 'Needs perfectly divisible fold on TrCV set of size {}'.format(len(tc))
    tc_label = [split_label(tc.index, n_folds, r) for r in range(repeat)]
    #
    indexify = lambda a: index.isin(set(a)) if isinstance(index, pd.Series) else pd.Series(index.isin(set(a)), index=index)
    repr = 'split(a={},f={},r={},h={},v={})'.format('f', n_folds, repeat, hoint, int(reverse))
    s = O(type='f', repeat=repeat, n_folds=n_folds, repr=repr)
    s.tc = indexify(tc)
    s.trcv = []
    for r in range(repeat):
        s.trcv += [(indexify(tc[tc_label[r] != f]), indexify(tc[tc_label[r] == f])) for f in range(n_folds)]
    if n_folds == 2 and repeat == 1:
        s.tr, s.cv = s.trcv[0][0], s.trcv[0][1]
    s.ho = indexify(ho)
    s.lb = indexify(days_z)
    s.aw = dh.Day > -1
    return s